# Reading Notes

In [38]:
import numpy as np

### Section 2 - Comparisons on two Examples

Below are implemented all algorithms mentioned in chapter 1. They are all realted to the product function
$$f:\mathbb{R}^n \to \mathbb{R}, \quad f(x_1,...,x_n) \mapsto \prod_{i=1}^n x_i$$
which has derivative
$$g := f^\prime:\mathbb{R}^n \to \mathbb{R^n}, \quad f(x_1,...,x_n) \mapsto \sum_{j=1}^n e_j\prod_{i\neq j}^n x_i.$$


In [37]:
# Evaluation of Product [p.7] 
# note idexing must be shifted back by one since the paper indexes from 1, but python indexes from 0

# setup
x = [1,3,3,2,7]
n = len(x)
x += [None] * n

# START OF ALGORITHM
x[n] = x[0]
for i in range(n+1, 2*n):
    x[i] = x[i - n] * x[i - 1]
y = x[2*n-1]
# END OF ALGORITHM

print(x)
print(y)
print(1*3*3*2*7)
print("yay! the algorithm works!")

[1, 3, 3, 2, 7, 1, 3, 9, 18, 126]
126
126
yay, the algorithm works!


In [49]:
# Forward Differentiation of Product [p.7] 

x = [2,3,5]
n = len(x)
x += [None] * n
dx = [None] * (2 * n) # defining unnecessarily long array so that inexing is more consistent with paper.

def e(i):
    return np.eye(1, n, i)

# START OF ALGORITHM
x[n] = x[0]
dx[n] = e(0)
for i in range(n+1, 2*n):
    x[i] = x[i - n] * x[i - 1]
    dx[i] = x[i - 1] * e(i - n) + x[i - n] * dx[i - 1]
y = x[2*n-1]
g = dx[2*n-1]
# END OF ALGORITHM

print(y)
print(g)
print("yay! algorithm works as expected!")

30
[[15. 10.  6.]]


In [1]:
# Reverse Differentiation of Product [p.8] 

x = [2,3,5]
n = len(x)
x += [None] * n
x_bar = [None] * (2 * n) # defining unnecessarily long array so that inexing is more consistent with paper.

def e(i):
    return np.eye(1, n, i)

# START OF ALGORITHM
x[n] = x[0]
for i in range(n+1, 2*n):
    x[i] = x[i - n] * x[i - 1] # forward step
y = x[2 * n - 1]

x_bar[2 * n - 1] = 1
for i in range(2*n - 1, n, -1):
    x_bar[i - 1] = x_bar[i] * x[i - n] # reverse step
    x_bar[i - n] = x_bar[i] * x[i - 1]
x_bar[0] = x_bar[n]
g = x_bar[0:n]
# END OF ALGORITHM

print("y =", y)
print(g)
print("yay! algorithm works as expected!")

y = 30
[15, 10, 6]
yay! algorithm works as expected!


### Section 3 - Automatic Differentiation on Composite Functions


By using a tree structure we implicitly have a similar memory layout to that of x_1, ..., x_n.

#### Backward Accumulation
The following node structure has accompanying functions for backward mode differentiation with partial derivatives.

future plans: expression class should have a member self.partial, an n-dim vector (where we have f:R^n -> R) with elements x1, ..., xn corresponding to d self.eval() / xi.

In [3]:
import numpy as np

# Node Classes

# Base Expression
# the _b at the end means it supports backward mode
class expression_b:
    def __init__(self):
        self.partial = 0
        self.value = 0

    def __str__():
        return "EXPRESSION" 

    def eval(self):
        self.partial = 0
        return "CANT EVALUATE BASE EXPRESSION CLASS"

    def derive():
        return "CANT DERIVE OF BASE EXPRESSION CLASS"

# Base Expression Types
    
class Un_Op_b(expression_b):
    def __init__(self, a):
        expression_b.__init__(self)
        self.a = a
    def __str__(self):
        return "[UNARY OP]"

class Bi_Op_b(expression_b):
    def __init__(self, a, b):
        expression_b.__init__(self)
        self.a = a
        self.b = b
    def __str__(self):
        return "[BINARY OP]"
    
# Usable Expression Types

# Constants
class Const_Exp_b(Un_Op_b):
    def __str__(self):
        return str(self.a)
    
    def eval(self):
        self.value = self.a
        return self.a
    def derive(self):
        return

# Variables  
class Var_b(expression_b):
    def __init__(self, name, value = None):
        '''
        name  : typeof(string)
        value : typeof(float)
        '''
        self.name = name
        self.value = value
        self.partial = 0
    def __str__(self):
        return self.name
    def eval(self):
        return self.value
    def derive(self):
        return
    
# Unary Operations
class Sin_Op_b(Un_Op_b):
    def __str__(self):
        return f"sin({str(self.a)})"
    def eval(self):
        self.value = np.sin(self.a.eval())
        return self.value
    def derive(self):
        dthis_da = np.cos(self.a.value)
        self.a.partial += dthis_da * self.partial
        self.a.derive()
        return
    
class Exp_Op_b(Un_Op_b):
    def __str__(self):
        return f"exp({str(self.a)})"
    def eval(self):
        self.value = np.exp(self.a.eval())
        return self.value
    def derive(self):
        dthis_da = np.exp(self.a.value)
        self.a.partial += dthis_da * self.partial
        self.a.derive()
        return

# Binary Operations
class Add_Op_b(Bi_Op_b):
    def __str__(self):
        return f"({str(self.a)} + {str(self.b)})"
    def eval(self):
        self.value = self.a.eval() + self.b.eval()
        return self.value
    def derive(self):
        dthis_da = 1
        dthis_db = 1
        self.a.partial += dthis_da * self.partial
        self.b.partial += dthis_db * self.partial
        self.a.derive()
        self.b.derive()
        return         
    
class Mult_Op_b(Bi_Op_b):
    def __str__(self):
        return f"({str(self.a)} * {str(self.b)})"  
    def eval(self):
        self.value = self.a.eval() * self.b.eval()
        return self.value
    def derive(self):
        dthis_da = self.b.value
        dthis_db = self.a.value
        self.a.partial += dthis_da * self.partial
        self.b.partial += dthis_db * self.partial
        self.a.derive()
        self.b.derive()
        return

In [4]:

x1 = Var_b("x1", 2)
x2 = Var_b("x2", 5)
x3 = Var_b("x3", -1)

x = [x1, x2, x3]
    
func = Sin_Op_b(Add_Op_b(x1, Mult_Op_b(Const_Exp_b(5), x2)))
print(f"f({[str(xi) for xi in x]}) = " + str(func))

for xi in x:
    print(f"Let {xi.name} = {xi.value}")

f_x = func.eval()

print(f"(f({[str(xi) for xi in x]})) = ", f_x)

func.partial = 1
func.derive()

for xi in x:
    print(f"df/d{xi.name} = {xi.partial}")


f(['x1', 'x2', 'x3']) = sin((x1 + (5 * x2)))
Let x1 = 2
Let x2 = 5
Let x3 = -1
(f(['x1', 'x2', 'x3'])) =  0.956375928404503
df/dx1 = -0.2921388087338362
df/dx2 = -1.4606940436691809
df/dx3 = 0


Testing backward mode differentiation on non-tree acyclic computational digrpahs

In [20]:
x1 = Var_b("x1", 2)
x2 = Var_b("x2", 2)
    
intermediate1 = Sin_Op_b(x1)
intermediate2 = Sin_Op_b(x2)
# these 2 functions should be the same, because we are just swapping the arguments of multiplication operation
func1 = Add_Op_b(Exp_Op_b(intermediate1), Sin_Op_b(intermediate1))
func2 = Add_Op_b(Sin_Op_b(intermediate2), Exp_Op_b(intermediate2))

print(f"f1({str(x1)}) = " + str(func1))
print(f"Let {x1.name} = {x1.value}")

f1_x = func1.eval()

print(f"(f1({str(x1)})) = ", f1_x)

func1.partial = 1
func1.derive()

print(f"df1/d{x1.name} = {x1.partial}")


#### for func2(x2)
print("")

print(f"f2({str(x2)}) = " + str(func2))
print(f"Let {x2.name} = {x2.value}")

f2_x = func2.eval()

print(f"(f2({str(x2)})) = ", f2_x)

func2.partial = 1
func2.derive()

print(f"df2/d{x2.name} = {x2.partial}")

f1(x1) = (exp(sin(x1)) + sin(sin(x1)))
Let x1 = 2
(f1(x1)) =  3.271650071587889
df1/dx1 = -2.3218728550844605

f2(x2) = (sin(sin(x2)) + exp(sin(x2)))
Let x2 = 2
(f2(x2)) =  3.271650071587889
df2/dx2 = -1.544395106181417


As suspected, the partial derivatives do not agree, since the version using `x1` calculated the partial of `x1` using the not yet fully known partial of `intermediate`.

We now aim to find an efficient method of ordering the tree such that left-first derivation is always correct. I don't yet know if this is possible, but i suspect it is. If it is possible, it could be done at the parser level, but this is not ideal, as for machine learning applicaions the function tree is built more explicitly and is not parsed from a string.

Possible aproaches to implementing backwards mode : <br>
 (a) parser-level graph ordering for left-first evaluation <br>
 (b) graph reordering for left-first evaluation <br>
 (c) non-left-first evaluation (perhaps something like breadth-first evaluation) <br>

(c) sounds most promising as it would not require an extra preprocessing step, whereas (a), (b) would.